## Introduction to modelling with Alea-carta-est

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artefactory/choice-learn/blob/main/notebooks/basket_models/alea_carta.ipynb)

We use a synthetic dataset to demonstrate how to use the Alea-carta-est model [1]

In [ ]:
# Install necessary packages that are not already installed in the environment
!pip install matplotlib

In [ ]:
import os
import sys

sys.path.append("../../")
print(os.getcwd())

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from choice_learn.basket_models import Trip, TripDataset
from choice_learn.basket_models.alea_carta import AleaCarta


### Generate a dataset with an interaction matrix

**Items:**
- Complementary pairs of items (the probability of these pairs should increase during training): 
    - $1$ / $2$ / $3$ & $4$ / $5$ / $6$
- Substitutable items:
    - $1$ & $2$ & $3$
    - $4$ & $5$ & $6$
- (Optionally) other items without special relations:
    - $7$, $8$, ...

**Concrete example:**
- Item $1$, $2$, $3$ = pastas
- Item $4$, $5$, $6$ = sauces
- Item $7$, $8$ = water and bread

**3 assortments:**
- Training set:
    - Store 1: $\mathcal{A}_1$ = all items except $3$ & $6$
    - Store 2: $\mathcal{A}_2$ = all items except $2$ & $4$
    - Store 3: $\mathcal{A}_3$ = all items except $1$ & $5$
- Test set:
    - Store 4: $\mathcal{A}_4$ = all items except $3$ & $4$

We generate the baskets by sampling from an interaction matrix following a given distribution.

    

In [ ]:
n_items = 8
n_stores = 4

First, we create an interaction matrix.

In [ ]:
def create_interaction_matrix(n: int, single_option: bool = False) -> np.ndarray:
    """Generate a random interaction matrix of size n x n.
    
    The matrix is symmetric and the diagonal is filled with zeros.
    The matrix is divided into two parts:
    - Complementary pairs (positive values)
    - Substitutable pairs (negative values)

    Parameters
    ----------
    n: int
        Size of the square matrix
    single_option: bool, optional
        If True, add a column representing single item options
        By default False
    
    Returns
    -------
    matrix: np.ndarray
        The interaction matrix
    """
    matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i + 1):
            if (i in [3, 4, 5] and j in [0, 1, 2]) or (i in [0, 1, 2] and j in [3, 4, 5]):
                # Complementary pair: >= 0
                # Draw a sample from the Gaussian distribution
                matrix[i, j] = np.random.normal(2, 1)
            elif (i in [3, 4, 5] and j in [3, 4, 5]) or (i in [0, 1, 2] and j in [0, 1, 2]):
                # Substitutable pair: <= 0
                # Draw a sample from the Gaussian distribution
                matrix[i, j] =  - 1 * np.random.normal(2, 1)
            else:
                # No interaction: 0 with small Gaussian noise
                matrix[i, j] = np.random.normal(loc=0, scale=0.05)
            
            # Copy the lower triangle to the upper triangle
            matrix[j, i] = matrix[i, j]

        # Same item
        matrix[i, i] = 0
    
    if single_option:
        # Add a column to the right for single item options
        matrix = np.hstack((matrix, np.random.normal(1, 0.5, (n, 1))))
    
    return matrix

In [ ]:
interaction_matrix_single_option = create_interaction_matrix(n_items, single_option=True)

print(f"Shape of the interaction matrix: {interaction_matrix_single_option.shape}\n")
print(f"{interaction_matrix_single_option=}")

In [ ]:
def plot_interaction_matrix(matrix: np.ndarray, half: bool = False) -> None:
    """Plot the interaction matrix.
    
    The matrix is displayed with a color map and the value
    of each cell is displayed inside the cell.

    Parameters
    ----------
    matrix: np.ndarray
        The interaction matrix to plot
    half: bool, optional
        If True, only the lower half of the matrix is displayed
        By default False

    Returns
    -------
    matrix: np.ndarray
        The interaction matrix
    """
    if half:
        # Mask: elements above the k-th diagonal are set to False, rest to True
        mask = ~np.triu(np.ones_like(matrix, dtype=bool), k=0)
        matrix = np.ma.array(matrix, mask=mask)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(matrix, cmap='coolwarm', interpolation='none', vmin=-1, vmax=1)
    plt.colorbar(label='Interaction Value')
    plt.title('Interaction Matrix')
    plt.xlabel('Item B')
    plt.ylabel('Item A')

    if matrix.shape[1] == matrix.shape[0]:
        # No single item option
        plt.xticks(ticks=np.arange(matrix.shape[0]), labels=np.arange(1, matrix.shape[0] + 1))
        plt.yticks(ticks=np.arange(matrix.shape[1]), labels=np.arange(1, matrix.shape[1] + 1))
    
    else:
        # Add a column to the right for single item options
        plt.xticks(ticks=np.arange(matrix.shape[0] + 1), labels=np.append(np.arange(1, matrix.shape[1]), "single"))
        plt.yticks(ticks=np.arange(matrix.shape[1]), labels=np.arange(1, matrix.shape[1] + 1))

    # Display the value inside each case
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if not half or (half and j >= i):
                plt.text(j, i, f'{matrix[i, j]:.2f}', ha='center', va='center', color='black')
    
    plt.show()

In [ ]:
plot_interaction_matrix(interaction_matrix_single_option, half=True)

In [ ]:
plot_interaction_matrix(interaction_matrix_single_option, half=False)

In [ ]:
def generate_pairs(
    interaction_matrix: np.ndarray,
    assortment: np.ndarray,
    num_baskets: int
) -> list:
    """Generate pairs of items based on the interaction matrix.

    Size of the baskets: 2

    Parameters
    ----------
    interaction_matrix: np.ndarray
        The interaction matrix to plot
    assortment: np.ndarray
        The assortment of items available in the store
    num_baskets: int
        The number of baskets to generate

    Returns
    -------
    baskets: list
        List of generated baskets
    """
    n_items = interaction_matrix.shape[0]

    # The interaction matrix should have a shape of (n_items, n_items) to generate pairs of items
    if interaction_matrix.shape[1] == n_items + 1:
        # Single item option
        print("Interaction matrix of shape (n_items, n_items + 1), generating baskets of varying size 1 or 2.")
        return generate_baskets(interaction_matrix, assortment, num_baskets)
    elif interaction_matrix.shape[1] != n_items:
        raise ValueError(
            "The interaction matrix must have a shape of (n_items, n_items + 1) "
            "or (n_items, n_items)."
        )

    # Build the list of available items from the assortment
    available_items = np.array([item_id for item_id in range(interaction_matrix.shape[0]) if assortment[item_id] == 1])
    
    baskets = []
    for _ in range(num_baskets):
        # Distribution for the first item with weights sampled from an uniform distribution
        normal_dist = np.random.uniform(0, 1, size=len(available_items))
        normal_dist /= np.sum(normal_dist)
        item_a = np.random.choice(
            available_items,
            p=normal_dist
        )

        # Get probabilities of the next item given the previous item
        # by applying the softmax function to the interaction values
        # of the row corresponding to the previous item
        probabilities = np.exp(interaction_matrix[item_a, :])
        probabilities /= probabilities.sum()
        item_b = None
        while (item_b is None) or (item_b == item_a) or (item_b not in available_items):
            # No duplicate items
            item_b = np.random.choice(n_items, p=probabilities)
        
        baskets.append(np.array([item_a, item_b]))

    return baskets


def generate_baskets(
    interaction_matrix: np.ndarray,
    assortment: np.ndarray,
    num_baskets: int
) -> list:
    """Generate baskets based on the interaction matrix.

    Size of the baskets: 1 or 2

    Parameters
    ----------
    interaction_matrix: np.ndarray
        The interaction matrix to plot
    assortment: np.ndarray
        The assortment of items available in the store
    num_baskets: int
        The number of baskets to generate

    Returns
    -------
    baskets: list
        List of generated baskets
    """
    n_items = interaction_matrix.shape[0]

    # The interaction matrix should have a shape of (n_items, n_items + 1)
    # to include the single item option
    if interaction_matrix.shape[1] == n_items:
        # No single item option
        print("Square interaction matrix, generating pairs instead of baskets of varying size 1 or 2.")
        return generate_pairs(interaction_matrix, assortment, num_baskets)
    elif interaction_matrix.shape[1] != n_items + 1:
        raise ValueError(
            "The interaction matrix must have a shape of (n_items, n_items + 1) "
            "or (n_items, n_items)."
        )

    # Build the list of available items from the assortment
    available_items = np.array([item_id for item_id in range(interaction_matrix.shape[0]) if assortment[item_id] == 1])
    available_items_with_single_option = np.append(available_items, n_items)

    # Distribution for the first item with weights sampled from an uniform distribution
    normal_dist = np.random.uniform(0, 1, size=len(available_items))
    normal_dist /= np.sum(normal_dist)
    
    baskets = []
    for _ in range(num_baskets):
        item_a = np.random.choice(
            available_items,
            p=normal_dist
        )

        # Get probabilities of the next item given the previous item
        # by applying the softmax function to the interaction values
        # of the row corresponding to the previous item
        # Shape: (n_items + 1,) (because of the single option added at the end)
        probabilities = np.exp(interaction_matrix[item_a, :])
        probabilities /= probabilities.sum()
        item_b = None
        while (item_b is None) or (item_b == item_a) or (item_b not in available_items_with_single_option):
            # No duplicate items
            item_b = np.random.choice(n_items + 1, p=probabilities)

        if item_b == n_items:
            # Single option
            baskets.append(np.array([item_a]))
        else:
            baskets.append(np.array([item_a, item_b]))

    return baskets


Then, we set the assortment of items available in each store.

In [ ]:
a0, a1, a2, a3, a4 = np.ones(n_items), np.ones(n_items), np.ones(n_items), np.ones(n_items), np.ones(n_items)
a1[0], a1[3] = 0, 0
a2[1], a2[4] = 0, 0
a3[2], a3[5] = 0, 0
a4[3], a4[2] = 0, 0

available_items = np.array([a0, a1, a2, a3, a4])

print(f"{available_items=}\n")
for i, assortment in enumerate(available_items):
    print(f"Assortment {i}: {assortment}, ie items {np.where(assortment == 1)[0] + 1} are available")

Now, we generate pairs of items and baskets using the interaction matrix.

In [ ]:
# num_baskets = 10_000
num_baskets = 100

purchases_assortment_0 = generate_baskets(
    interaction_matrix=interaction_matrix_single_option,
    assortment=available_items[0],
    # All baskets with assortment 0 (ie all items / no assortment taken into account)
    num_baskets=num_baskets,
)
purchases_assortment_1 = generate_baskets(
    interaction_matrix=interaction_matrix_single_option,
    assortment=available_items[1],
    # Half of the baskets with assortment 1
    num_baskets=num_baskets // 3,
)
purchases_assortment_2 = generate_baskets(
    interaction_matrix=interaction_matrix_single_option,
    assortment=available_items[2],
    # Half of the baskets with assortment 2
    num_baskets=num_baskets // 3,
)
purchases_assortment_3 = generate_baskets(
    interaction_matrix=interaction_matrix_single_option,
    assortment=available_items[3],
    # All the baskets with assortment 3
    num_baskets=num_baskets // 3,
)
purchases_assortment_4 = generate_baskets(
    interaction_matrix=interaction_matrix_single_option,
    assortment=available_items[4],
    # All the baskets with assortment 3
    num_baskets=num_baskets,
)

In [ ]:
print(f"{len(purchases_assortment_0)=}, {len(purchases_assortment_1)=}, {len(purchases_assortment_2)=}, {len(purchases_assortment_3)=}, {len(purchases_assortment_4)=}\n")

print(f"With all items available: {purchases_assortment_0[:10]=}")
print(f"From assortment without 3 & 6: {purchases_assortment_1[:10]=}")
print(f"From assortment without 2 & 4: {purchases_assortment_2[:10]=}")
print(f"From assortment without 1 & 5: {purchases_assortment_3[:10]=}\n")
print(f"From assortment without 3 & 4: {purchases_assortment_4[:10]=}\n")

min_length_purchases_assortment_0, max_length_purchases_assortment_0 = min(len(basket) for basket in purchases_assortment_0), max(len(basket) for basket in purchases_assortment_0)
print(f"Minimum and maximum lengths of arrays in purchases_assortment_0: {min_length_purchases_assortment_0} & {max_length_purchases_assortment_0}")
min_length_purchases_assortment_1, max_length_purchases_assortment_1= min(len(basket) for basket in purchases_assortment_1), max(len(basket) for basket in purchases_assortment_1)
print(f"Minimum and maximum lengths of arrays in purchases_assortment_1: {min_length_purchases_assortment_1} & {max_length_purchases_assortment_1}")
min_length_purchases_assortment_2, max_length_purchases_assortment_2 = min(len(basket) for basket in purchases_assortment_2), max(len(basket) for basket in purchases_assortment_2)
print(f"Minimum and maximum lengths of arrays in purchases_assortment_2: {min_length_purchases_assortment_2} & {max_length_purchases_assortment_2}")
min_length_purchases_assortment_3, max_length_purchases_assortment_3 = min(len(basket) for basket in purchases_assortment_3), max(len(basket) for basket in purchases_assortment_3)
print(f"Minimum and maximum lengths of arrays in purchases_assortment_3: {min_length_purchases_assortment_3} & {max_length_purchases_assortment_3}")
min_length_purchases_assortment_4, max_length_purchases_assortment_4 = min(len(basket) for basket in purchases_assortment_4), max(len(basket) for basket in purchases_assortment_4)
print(f"Minimum and maximum lengths of arrays in purchases_assortment_4: {min_length_purchases_assortment_4} & {max_length_purchases_assortment_4}")

Now, we can create trips from the generated baskets.

In [ ]:
store = 0
week = 0
prices = np.arange(1, n_items + 1) * 1.1

In [ ]:
trip_list_train_assortment_1 = list(np.concatenate(
    [
        [Trip(purchases=basket, store=store, week=week, prices=prices, assortment=1)]
        for basket in purchases_assortment_1
    ]
))
trip_list_train_assortment_2 = list(np.concatenate(
    [
        [Trip(purchases=basket, store=store, week=week, prices=prices, assortment=2)]
        for basket in purchases_assortment_2
    ]
))
trip_list_train_assortment_3 = list(np.concatenate(
    [
        [Trip(purchases=basket, store=store, week=week, prices=prices, assortment=3)]
        for basket in purchases_assortment_3
    ]
))
trip_list_train = trip_list_train_assortment_1 + trip_list_train_assortment_2 + trip_list_train_assortment_3

# Trips with baskets from purchases_assortment_3 and with assortment
trip_list_test = list(np.concatenate(
    [
        [Trip(purchases=basket, store=store, week=week, prices=prices, assortment=4)]
        for basket in purchases_assortment_4
    ]
))

We group the trips into a ```TripDataset``` object.

In [ ]:
trip_dataset_train = TripDataset(trips=trip_list_train, available_items=available_items)
trip_dataset_test = TripDataset(trips=trip_list_test, available_items=available_items)

print(f"{len(trip_dataset_train)=}\n{len(trip_dataset_test)=}")

In [ ]:
n_items_train, n_stores_train = trip_dataset_train.n_items, trip_dataset_train.n_stores
n_items_test, n_stores_test = trip_dataset_test.n_items, trip_dataset_test.n_stores

We instantiate an Alea-carta-est model and fit it to the training data.

In [ ]:
latent_sizes = {"preferences": 6, "price": 3, "season": 3}
n_negative_samples = 3
optimizer = "adam"
lr = 1e-2
epochs = 200
# epochs = 1000
batch_size = 32

In [ ]:
model = AleaCarta(
    # item_intercept=True,
    item_intercept=False,
    price_effects=False,
    seasonal_effects=False,
    latent_sizes=latent_sizes,
    n_negative_samples=n_negative_samples,
    optimizer=optimizer,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

model.instantiate(n_items=n_items, n_stores=n_stores)

In [ ]:
history = model.fit(trip_dataset=trip_dataset_train)

In [ ]:
print(history.keys())

In [ ]:
plt.plot(history["train_loss"])
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.legend()
plt.title("Training of Shopper")
plt.show()

In [ ]:
model.compute_batch_utility(item_batch=np.array(list(range(2))),
        basket_batch=np.array([[1, 2], [3, 4]]),
        store_batch=np.array([0, 0]),
        week_batch=np.array([0, 0]),
        price_batch=np.array([[0, 0], [0, 0]])
        )

### References
[1] Better Capturing Interactions between Products in Retail: Revisited Negative Sampling for Basket Choice Modeling, Désir, J.; Auriaut, V.; Možina, M.; Malherbe, E. (2025), ECML PKDDD Applied Data Science